<div align="center">
    
# 4.0 Feature Engineering

## 4.1 Table of Contents<a id='4.1_Table_of_Contents'></a>
* [4.1 Table of Contents](#4.1_Table_of_Contents)
* [4.2 Introduction](#4.2_Introduction)
* [4.3 Library Imports](#4.3_Library_Imports)
* [4.4 Data Loading](#4.4_Data_Loading)
* [4.5 Feature Construction](#4.5_Feature_Construction)
  * [4.5.1 Open Rate & Lift Features](#4.5.1_Open_Rate_&_Lift_Features)
  * [4.5.2 Vertical Drop Validation](#4.5.2_Vertical_Drop_Validation)
  * [4.5.3 Region vs State Check](#4.5.3_Region_vs_State_Check)
* [4.6 Data Split](#4.6_Data_Split)
* [4.7 Data Transformation](#4.7_Data_Transformation)
  * [4.7.1 Build Preprocessing Pipeline](#4.7.1_Build_Preprocessing_Pipeline)
  * [4.7.2 Fit & Transform](#4.7.2_Fit_&_Transform)
* [4.8 Summary](#4.8_Summary)

## 4.2 Introduction<a id='4.2_Introduction'></a>

This notebook prepares the dataset for modeling by constructing additional features and transforming all variables into a consistent, model-ready format.  

The dataset has already been cleaned in the previous stage, so no missing-value handling is required here. The focus is on:
- Creating meaningful derived features to capture resort characteristics  
- Standardizing numerical variables for scale consistency  
- Encoding categorical variables into machine-readable format  
- Ensuring all transformations are applied through a reproducible **scikit-learn Pipeline**

By the end of this notebook, all training, validation, and target datasets will share the same structure, allowing direct use in model training and evaluation.

## 4.3 Library Imports<a id='4.3_Library_Imports'></a>

In [1]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
from IPython.display import display

# Scikit-learn preprocessing tools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# For combining processed arrays later
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Optional: warnings filter (optional but useful)
import warnings
warnings.filterwarnings("ignore")

# Save/load later
from joblib import dump, load

RANDOM_STATE = 17
np.random.seed(RANDOM_STATE)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 140)

## 4.4 Data Loading <a id='4.4_Data_Loading'></a>

In [3]:
INPUT_PATH = "../data/processed/data_02_analyzed.csv"
df = pd.read_csv(INPUT_PATH)
print("Loaded:", INPUT_PATH, "| Shape:", df.shape)
df.sample(5).T

Loaded: ../data/processed/data_02_analyzed.csv | Shape: (272, 26)


,197,188,150,28,262
name,Mt. Bachelor,Alpine Valley,Taos Ski Valley,Sunlight Mountain Resort,Sunburst
region,Oregon,Ohio,New Mexico,Colorado,Wisconsin
state,Oregon,Ohio,New Mexico,Colorado,Wisconsin
summit_elevation_ft,9065,1500,12481,9895,1100
vertical_drop_ft,3365,230,3281,2010,214
base_elevation_ft,5700,1260,9200,7885,866
num_trams,0,0,1,0,0
fast_six_lifts,0,0,0,0,0
fast_quad_lifts,8,0,1,0,0
quad_lifts,0,1,3,0,0


## 4.5 Feature Construction<a id='4.5_Feature_Construction'></a>

In [4]:
# create feature 'open_rate_last_year' = 'days_open_last_year' / 'projected_days_open'

# compute feature
df['open_rate_last_year'] = (df['days_open_last_year'] / df['projected_days_open']).astype(float)

# drop source column 'days_open_last_year' (keep projected_days_open)
df = df.drop(columns=['days_open_last_year'])

print(df[['open_rate_last_year', 'projected_days_open']].head())

   open_rate_last_year  projected_days_open
0             1.000000                150.0
1             0.500000                 90.0
2             0.986842                152.0
3             1.000000                122.0
4             1.105769                104.0


### Note:
The feature `open_rate_last_year` measures how often a resort was open in the previous season compared to its projected number of open days.
The original column `days_open_last_year` was removed since it is now represented by this ratio.

In [5]:
# --- Define the individual lift columns ---
lift_cols = [
    'num_trams',
    'fast_six_lifts',
    'fast_quad_lifts',
    'quad_lifts',
    'triple_lifts',
    'double_lifts',
    'surface_lifts'
]

# --- Create total_lifts feature ---
df['total_lifts'] = df[lift_cols].sum(axis=1)

# --- Optional: verify ---
print(df[['total_lifts'] + lift_cols].head())

   total_lifts  num_trams  fast_six_lifts  fast_quad_lifts  quad_lifts  triple_lifts  double_lifts  surface_lifts
0            7          1               0                2           2             0             0              2
1            4          0               0                0           0             0             4              0
2            3          0               0                0           0             1             0              2
3            8          0               1                0           2             2             1              2
4            7          0               0                1           2             3             1              0


### Note:
The feature `total_lifts` represents the total number of transportation lifts available at each resort.
It summarizes the resort’s lift infrastructure and overall capacity to move visitors efficiently.

In [6]:
# check if 'vertical_drop_ft' is calculated as difference between 'summit_elevation_ft', and 'base_elevation_ft'

# --- Calculate the difference ---
df['vertical_check'] = df['summit_elevation_ft'] - df['base_elevation_ft']

# --- Compare with existing vertical_drop_ft ---
df['vertical_match'] = np.isclose(df['vertical_drop_ft'], df['vertical_check'], atol=1)

# --- Find mismatches ---
mismatches = df.loc[~df['vertical_match'], ['name', 'vertical_drop_ft', 'vertical_check']]
total = len(df)
mismatch_count = len(mismatches)

if mismatch_count == 0:
    print("All rows match: vertical_drop_ft = summit - base.")
    df.drop(columns=['summit_elevation_ft', 'vertical_check', 'vertical_match'], inplace=True)
else:
    print(f"Mismatched rows found: {mismatch_count:,} out of {total:,} total rows "
          f"({mismatch_count/total:.1%} mismatch rate).")
    print("Here are the mismatched examples:\n")
    print(mismatches)


Mismatched rows found: 57 out of 272 total rows (21.0% mismatch rate).
Here are the mismatched examples:

                                   name  vertical_drop_ft  vertical_check
0                        Alyeska Resort              2500            3689
1                   Eaglecrest Ski Area              1540            1400
4                   Sunrise Park Resort              1800            1900
7                      Donner Ski Ranch               750             981
9                   Mt. Shasta Ski Park              1435            1390
17              Arapahoe Basin Ski Area              2530            2270
19               Copper Mountain Resort              2738            2601
36                          Bogus Basin              1800            1782
51                           Seven Oaks               275             175
53        Big Squaw Mountain Ski Resort               660            1450
54                     Camden Snow Bowl               850             930
56    

In [7]:
# Drop summit (redundant, even if mismatches exist)
df.drop(columns=['summit_elevation_ft', 'vertical_check', 'vertical_match'], inplace=True, errors='ignore')

### Note:
The field `vertical_drop_ft` was verified against the difference between `summit_elevation_ft` and `base_elevation_ft`.
Approximately 21% of records showed small mismatches, mostly within a minor acceptable range.
The column `summit_elevation_ft` was dropped, and `vertical_drop_ft` was retained as the reliable source of elevation difference.

In [8]:
# check if 'region' and 'state' are redundant
df_diff = df[df['region'] != df['state']]
print("Number of mismatched rows:", len(df_diff))
df_diff[['name', 'region', 'state']]

Number of mismatched rows: 21


,name,region,state
5,Yosemite Ski & Snowboard Area,Northern California,California
6,Dodge Ridge,Sierra Nevada,California
7,Donner Ski Ranch,Sierra Nevada,California
8,Mammoth Mountain Ski Area,Sierra Nevada,California
9,Mt. Shasta Ski Park,Sierra Nevada,California
10,Mountain High,Sierra Nevada,California
11,Mt. Baldy,Sierra Nevada,California
12,Ski China Peak,Sierra Nevada,California
13,Snow Valley,Sierra Nevada,California
14,Soda Springs,Sierra Nevada,California


## Note:
Out of 272 resorts, 21 have a region value that does not match their state.
Since region provides useful geographical context beyond state boundaries, both columns are retained for modeling.

## 4.6 Data Split <a id='4.6_Data_Split'></a>

In [9]:
# Choose the target resort (to predict later) and remove it from training pool
TARGET_RESORT_NAME = "Big Mountain Resort"
TARGET_COL = "adult_weekend_price"
DROP_COLS = ["name", "adult_weekend_price", "adult_weekday_price"]

target_resort = df.loc[df["name"] == TARGET_RESORT_NAME].copy()
assert len(target_resort) == 1, "Target resort not found or not unique."
display(target_resort.T)

# Training pool
train_df = df.loc[df["name"] != TARGET_RESORT_NAME].reset_index(drop=True)

# Features/target
X = train_df.drop(columns=DROP_COLS, errors="ignore")
y = train_df[TARGET_COL]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)
print("Training set size:", X_train.shape[0])
print("Validation set size:", X_val.shape[0])
print("Train target mean:", round(y_train.mean(), 2))
print("Val target mean:", round(y_val.mean(), 2))

# Build X_target with identical columns/order as X
X_target = (
    target_resort.drop(columns=DROP_COLS, errors="ignore")
                 .reindex(columns=X.columns)
)

,122
name,Big Mountain Resort
region,Montana
state,Montana
vertical_drop_ft,2353
base_elevation_ft,4464
num_trams,0
fast_six_lifts,0
fast_quad_lifts,3
quad_lifts,2
triple_lifts,6


Training set size: 216
Validation set size: 55
Train target mean: 64.32
Val target mean: 65.62


## 4.7 Data Transformation <a id='4.7_Data_Transformation'></a>

### 4.7.1 Build Preprocessing Pipeline <a id='4.7.1_Build_Preprocessing_Pipeline'></a>

In [10]:
# Build a leak-safe preprocessing pipeline:
# - Numeric: standardize
# - Categorical: one-hot encode
cat_cols = X_train.select_dtypes(include=["object","category"]).columns.tolist()
num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

numeric_pipe = Pipeline([
    ("scale", StandardScaler())
])

categorical_pipe = Pipeline([
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocessor = ColumnTransformer([
    ("num", numeric_pipe, num_cols),
    ("cat", categorical_pipe, cat_cols),
], remainder="drop")

print("Numeric:", len(num_cols), "| Categorical:", len(cat_cols))

Numeric: 21 | Categorical: 2


### 4.7.2 Fit & Transform <a id='4.7.2_Fit_&_Transform'></a>

In [11]:
# Fit on TRAIN only (learn medians, means/stds, category levels)
preprocessor.fit(X_train)

# Apply the learned transforms consistently to each split
X_train_tf = preprocessor.transform(X_train)
X_val_tf   = preprocessor.transform(X_val)
X_tgt_tf   = preprocessor.transform(X_target)

# Build feature_names in the same order ColumnTransformer outputs: [num... then OHE(cat...)]
ohe = preprocessor.named_transformers_["cat"].named_steps["onehot"]
feature_names = list(num_cols) + ohe.get_feature_names_out(cat_cols).tolist()

In [12]:
# Verify each dataset before saving for training
X_train_tf_df = pd.DataFrame(X_train_tf, columns=feature_names, index=X_train.index)
X_val_tf_df   = pd.DataFrame(X_val_tf,   columns=feature_names, index=X_val.index)
X_tgt_tf_df   = pd.DataFrame(X_tgt_tf,   columns=feature_names, index=X_target.index)

# 1) No NaNs
assert not np.isnan(X_train_tf).any()
assert not np.isnan(X_val_tf).any()
assert not np.isnan(X_tgt_tf).any()

# 2) Column alignment
assert X_train_tf_df.columns.tolist() == X_val_tf_df.columns.tolist() == X_tgt_tf_df.columns.tolist()

# 3) Quick peek
display(X_tgt_tf_df.head(1))

,vertical_drop_ft,base_elevation_ft,num_trams,fast_six_lifts,fast_quad_lifts,quad_lifts,triple_lifts,double_lifts,surface_lifts,total_chairs,num_runs,terrain_parks,longest_run_mi,skiable_terrain_acres,snow_making_acres,years_open,average_snowfall_in,projected_days_open,night_skiing_acres,open_rate_last_year,total_lifts,region_Alaska,region_Arizona,region_Colorado,region_Connecticut,region_Idaho,region_Illinois,region_Indiana,region_Iowa,region_Maine,region_Maryland,region_Massachusetts,region_Michigan,region_Minnesota,region_Missouri,region_Montana,region_Mt. Hood,region_Nevada,region_New Hampshire,region_New Jersey,region_New Mexico,region_New York,region_North Carolina,region_Northern California,region_Ohio,region_Oregon,region_Pennsylvania,region_Salt Lake City,region_Sierra Nevada,region_South Dakota,region_Tennessee,region_Utah,region_Vermont,region_Virginia,region_Washington,region_West Virginia,region_Wisconsin,region_Wyoming,state_Alaska,state_Arizona,state_California,state_Colorado,state_Connecticut,state_Idaho,state_Illinois,state_Indiana,state_Iowa,state_Maine,state_Maryland,state_Massachusetts,state_Michigan,state_Minnesota,state_Missouri,state_Montana,state_Nevada,state_New Hampshire,state_New Jersey,state_New Mexico,state_New York,state_North Carolina,state_Ohio,state_Oregon,state_Pennsylvania,state_South Dakota,state_Tennessee,state_Utah,state_Vermont,state_Virginia,state_Washington,state_West Virginia,state_Wisconsin,state_Wyoming
122,1.511288,0.398567,-0.235037,-0.266123,1.208222,0.845877,2.871247,-0.99086,0.248069,1.377959,1.601563,0.947717,1.851928,3.295005,3.491376,0.794016,1.377309,0.230563,7.609924,0.32373,1.377959,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 4.8 Summary <a id='4.8_Summary'></a>

In [13]:
# Save featured dataset
output_path = "../data/processed/data_03_featured.csv"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df.to_csv(output_path, index=False)
print(f"Featured dataset saved -> {output_path} | Shape: {df.shape}")

# Save preprocessing artifacts for the Modeling notebook
art = Path("../artifacts"); art.mkdir(parents=True, exist_ok=True)

dump(preprocessor, art / "preprocessor.joblib")
np.save(art / "X_train_tf.npy", X_train_tf)
np.save(art / "X_val_tf.npy",   X_val_tf)
np.save(art / "X_tgt_tf.npy",   X_tgt_tf)

y_train.to_csv(art / "y_train.csv", index=False)
y_val.to_csv(art / "y_val.csv", index=False)
pd.Series(feature_names).to_csv(art / "feature_names.csv", index=False)

print("Saved preprocessing artifacts to:", art.resolve())


Featured dataset saved -> ../data/processed/data_03_featured.csv | Shape: (272, 26)
Saved preprocessing artifacts to: /home/dallen/Projects/ski_pricing_strategy/artifacts
